In [2]:
import pandas as pd
import os
import requests
import json
import altair as alt
import numpy as np

In [2]:
api_key = "RGAPI-ea6f4e66-adb6-4295-9551-99053de9dee5"

In [6]:
puuid = "C5pz-r_LXeHapwqxflnPxizTjQ2HDYrwkRaa7plZshcRjfEk0SZv3--vFs1QBqevX8Zmw1tVffwZbg"

In [7]:
matches = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?type=ranked&start=0&count=20&api_key={api_key}").json()

In [28]:
matchId = matches[3]

matchId

'NA1_4940509479'

In [11]:
match_re = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}")
match = match_re.json()['info']

In [27]:
for part in match['participants']:
    print(part['puuid'], part['teamPosition'])

C5pz-r_LXeHapwqxflnPxizTjQ2HDYrwkRaa7plZshcRjfEk0SZv3--vFs1QBqevX8Zmw1tVffwZbg TOP
vN3B7EAE1itvVuQVPeW7aDaUCJCDcVpblPwJdaen7j3fDQUsBMrPQpzIL3pQDHd5Ilz0qE7TDZNFqw JUNGLE
O1nQwEeZFEKJ0_74thl26RCrlXgv_BCxLdIZtbiFS4Asly6Me2FJBHJU7on9qEYwmw0iPrm2qeA2Rg MIDDLE
PIccSmOYccgwxblc3t6kUUvCiESy_1_DljdJu5FDWgAfOWb6cYYVLROvpuDcUuHLP28FFE2vuxnB8Q BOTTOM
pi282pfvBCl2VcSvdWaY-RONtpZm8F1l--KHOX8JhR_A_aP3T86-37ZpLaKtqOQYI8sV0Rl3s9Dy_Q UTILITY
7BwBpKzdqv1TxHy6cax9hXX-tYCcd-m6wFyXskPA5anIpa6jGC0WUCt-ylqpwt3RWwXtWwECzUq-rQ TOP
ivODQFIdV1eJskfUW1XKu2ePheMi5dHFOqV7e3VyTPbSOeKcoWDONvARmqSpnBZh3kOFtoag_kvztQ JUNGLE
2FDC-fupPUvo6WNmM0fD-6K5HPz9xtCSTOA7ncUIaBY7lZfS88NhgxIQDsfwIG4oNDoWYRiIffFyOQ MIDDLE
u8Wzxwgtn1ZNUJP20J98VgdrqN4gS46--gEi5WiS7r7CqwYvVTCR3RRS33xs2Z3evbukPj_vkLvCGg BOTTOM
dSRMTc1ZLtZzXB6JC0zY28Bhb2zDDt_XhIX4X6unUqVUr7V6nB2XlzDWp8DszSr_OAnxSVb_Bvgufg UTILITY


In [25]:
match['participants'][0].keys()

dict_keys(['allInPings', 'assistMePings', 'assists', 'baronKills', 'basicPings', 'bountyLevel', 'challenges', 'champExperience', 'champLevel', 'championId', 'championName', 'championTransform', 'commandPings', 'consumablesPurchased', 'damageDealtToBuildings', 'damageDealtToObjectives', 'damageDealtToTurrets', 'damageSelfMitigated', 'dangerPings', 'deaths', 'detectorWardsPlaced', 'doubleKills', 'dragonKills', 'eligibleForProgression', 'enemyMissingPings', 'enemyVisionPings', 'firstBloodAssist', 'firstBloodKill', 'firstTowerAssist', 'firstTowerKill', 'gameEndedInEarlySurrender', 'gameEndedInSurrender', 'getBackPings', 'goldEarned', 'goldSpent', 'holdPings', 'individualPosition', 'inhibitorKills', 'inhibitorTakedowns', 'inhibitorsLost', 'item0', 'item1', 'item2', 'item3', 'item4', 'item5', 'item6', 'itemsPurchased', 'killingSprees', 'kills', 'lane', 'largestCriticalStrike', 'largestKillingSpree', 'largestMultiKill', 'longestTimeSpentLiving', 'magicDamageDealt', 'magicDamageDealtToChampion

In [15]:
gold_time = pd.DataFrame(columns = ['Champion', 'Time', "Gold"])

In [11]:
def get_match_gold(matchId, puuid): #Given a matchId and a player's puuid, get their gold income over time for that game.
    time_re = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}/timeline?api_key={api_key}")
    match_re = requests.get(f"https://americas.api.riotgames.com/lol/match/v5/matches/{matchId}?api_key={api_key}")

    timeline = time_re.json()['info']
    match = match_re.json()['info']


    participant = pd.DataFrame(timeline['participants']).set_index('puuid').loc[puuid].get('participantId') #Get the participantId of the player with given puuid

    champ_names = {player['puuid'] : player['championName'] for player in match['participants']} #Get the names of everyone's champions

    times = [(champ_names[puuid], np.round(frame['timestamp'] / 60000, 0) , frame['participantFrames'][f'{participant}']['totalGold'])  #Get the timestamps and gold at each timestamp for given participantId
             for frame in timeline['frames']]
    
    return pd.DataFrame(times, columns = ['Champion', 'Time', 'Gold']) #Transform to DataFrame and return


In [16]:
for match in matches[:10]:
    gold_time = pd.concat([gold_time, get_match_gold(match, puuid)])

In [17]:
gold_time

,Champion,Time,Gold
0,Aatrox,0.0,500
1,Aatrox,1.0,500
2,Aatrox,2.0,584
3,Aatrox,3.0,885
4,Aatrox,4.0,1099
...,...,...,...
20,Aatrox,20.0,9522
21,Aatrox,21.0,10626
22,Aatrox,22.0,10922
23,Aatrox,23.0,11468


In [18]:
gold_time_final = gold_time.groupby(['Champion', 'Time'])['Gold'].mean().reset_index()

In [19]:
alt.Chart(gold_time_final).mark_line().encode(
    x = alt.X('Time'),
    y = alt.Y('Gold'),
    color = alt.Color('Champion')
)

alt.Chart(...)

In [68]:
filepath = os.path.join("champs.txt")
champs = ""
with open(filepath) as f:
    champs = f.read()

champs

"Aatrox\n\nAhri\n\nAkali\n\nAkshan\n\nAlistar\n\nAmumu\n\nAnivia\n\nAnnie\n\nAshe\n\nAurelion Sol\n\nAzir\n\nBard\n\nBel'Veth\n\nBlitzcrank\n\nBrand\n\nBraum\n\nBriar\n\nCaitlyn\n\nCamille\n\nCassiopeia\n\nCho'Gath\n\nCorki\n\nDarius\n\nDiana\n\nDr. Mundo\n\nDraven\n\nEkko\n\nElise\n\nEvelynn\n\nEzreal\n\nFiddlesticks\n\nFiora\n\nFizz\n\nGalio\n\nGangplank\n\nGaren\n\nGnar\n\nGragas\n\nGraves\n\nGwen\n\nHecarim\n\nHeimerdinger\n\nIllaoi\n\nIrelia\n\nIvern\n\nJanna\n\nJarvan IV\n\nJax\n\nJayce\n\nJhin\n\nJinx\n\nK'Sante\n\nKai'Sa\n\nKalista\n\nKarma\n\nKarthus\n\nKassadin\n\nKatarina\n\nKayle\n\nKayn\n\nKennen\n\nKha'Zix\n\nKindred\n\nKled\n\nKog'Maw\n\nLeBlanc\n\nLee Sin\n\nLeona\n\nLillia\n\nLissandra\n\nLucian\n\nLulu\n\nLux\n\nMalphite\n\nMalzahar\n\nMaokai\n\nMaster Yi\n\nMilio\n\nMiss Fortune\n\nMordekaiser\n\nMorgana\n\nNaafiri\n\nNami\n\nNasus\n\nNautilus\n\nNeeko\n\nNidalee\n\nNilah\n\nNocturne\n\nNunu & Willump\n\nOlaf\n\nOrianna\n\nOrnn\n\nPantheon\n\nPoppy\n\nPyke\n\nQiyana\

In [69]:
import re

In [70]:
champs_list = re.findall("[\w' &.]+", champs)

In [133]:
champs_list

champs_names_list = [
    "Wukong",
    "Nunu",
    "Renata",
    "AurelionSol",
    "Belveth",
    "Chogath",
    "DrMundo",
    "JarvanIV",
    "Kaisa",
    "Khazix",
    "KogMaw",
    "KSante",
    "LeeSin",
    "MasterYi",
    "MissFortune",
    "RekSai",
    "TahmKench",
    "TwistedFate",
    "XinZhao",
    "Aatrox",
    "Ahri",
    "Akali",
    "Akshan",
    "Alistar",
    "Amumu",
    "Anivia",
    "Annie",
    "Ashe",
    "Azir",
    "Bard",
    "Blitzcrank",
    "Brand",
    "Braum",
    "Briar",
    "Caitlyn",
    "Camille",
    "Cassiopeia",
    "Corki",
    "Darius",
    "Diana",
    "Draven",
    "Ekko",
    "Elise",
    "Evelynn",
    "Ezreal",
    "Fiddlesticks",
    "Fiora",
    "Fizz",
    "Galio",
    "Gangplank",
    "Garen",
    "Gnar",
    "Gragas",
    "Graves",
    "Gwen",
    "Hecarim",
    "Heimerdinger",
    "Illaoi",
    "Irelia",
    "Ivern",
    "Janna",
    "Jax",
    "Jayce",
    "Jhin",
    "Jinx",
    "Kalista",
    "Karma",
    "Karthus",
    "Kassadin",
    "Katarina",
    "Kayle",
    "Kayn",
    "Kennen",
    "Kindred",
    "Kled",
    "Leona",
    "Lillia",
    "Lissandra",
    "Lucian",
    "Lulu",
    "Lux",
    "Malphite",
    "Malzahar",
    "Maokai",
    "Milio",
    "Mordekaiser",
    "Morgana",
    "Naafiri",
    "Nami",
    "Nasus",
    "Nautilus",
    "Neeko",
    "Nidalee",
    "Nilah",
    "Nocturne",
    "Olaf",
    "Orianna",
    "Ornn",
    "Pantheon",
    "Poppy",
    "Pyke",
    "Qiyana",
    "Quinn",
    "Rakan",
    "Rammus",
    "Rell",
    "Renekton",
    "Rengar",
    "Riven",
    "Rumble",
    "Ryze",
    "Samira",
    "Sejuani",
    "Senna",
    "Seraphine",
    "Sett",
    "Shaco",
    "Shen",
    "Shyvana",
    "Singed",
    "Sion",
    "Sivir",
    "Skarner",
    "Sona",
    "Soraka",
    "Swain",
    "Sylas",
    "Syndra",
    "Taliyah",
    "Talon",
    "Taric",
    "Teemo",
    "Thresh",
    "Tristana",
    "Trundle",
    "Tryndamere",
    "Twitch",
    "Udyr",
    "Urgot",
    "Varus",
    "Vayne",
    "Veigar",
    "VelKoz",
    "Vex",
    "Vi",
    "Viktor",
    "Vladimir",
    "Volibear",
    "Warwick",
    "Xayah",
    "Xerath",
    "Yasuo",
    "Yone",
    "Yorick",
    "Yuumi",
    "Zac",
    "Zed",
    "Zeri",
    "Ziggs",
    "Zilean",
    "Zoe",
    "Zyra",
    "Hwei",
    "Smolder",
    "Leblanc",
    "None"
]

In [134]:
pd.DataFrame.to_json(pd.DataFrame(range(len(champs_list)), index = champs_list))

'{"0":{"Aatrox":0,"Ahri":1,"Akali":2,"Akshan":3,"Alistar":4,"Amumu":5,"Anivia":6,"Annie":7,"Ashe":8,"Aurelion Sol":9,"Azir":10,"Bard":11,"Bel\'Veth":12,"Blitzcrank":13,"Brand":14,"Braum":15,"Briar":16,"Caitlyn":17,"Camille":18,"Cassiopeia":19,"Cho\'Gath":20,"Corki":21,"Darius":22,"Diana":23,"Dr. Mundo":24,"Draven":25,"Ekko":26,"Elise":27,"Evelynn":28,"Ezreal":29,"Fiddlesticks":30,"Fiora":31,"Fizz":32,"Galio":33,"Gangplank":34,"Garen":35,"Gnar":36,"Gragas":37,"Graves":38,"Gwen":39,"Hecarim":40,"Heimerdinger":41,"Illaoi":42,"Irelia":43,"Ivern":44,"Janna":45,"Jarvan IV":46,"Jax":47,"Jayce":48,"Jhin":49,"Jinx":50,"K\'Sante":51,"Kai\'Sa":52,"Kalista":53,"Karma":54,"Karthus":55,"Kassadin":56,"Katarina":57,"Kayle":58,"Kayn":59,"Kennen":60,"Kha\'Zix":61,"Kindred":62,"Kled":63,"Kog\'Maw":64,"LeBlanc":65,"Lee Sin":66,"Leona":67,"Lillia":68,"Lissandra":69,"Lucian":70,"Lulu":71,"Lux":72,"Malphite":73,"Malzahar":74,"Maokai":75,"Master Yi":76,"Milio":77,"Miss Fortune":78,"Mordekaiser":79,"Morgana":8

In [135]:
from bs4 import BeautifulSoup

champ_counters = {}

for champ in champs_names_list:
    champ_name = champ.lower()
    print(champ)
    champ_counters[champ] = []
    request = requests.get(f"https://lolalytics.com/lol/{champ_name}/counters/").text
    soup = BeautifulSoup(request)


    for i in soup.findAll('span', attrs = {'class' : None, 'property' : None})[3:]:
        counter = re.search("[a-zA-Z &.']+", i.text)
        if(counter):
            if champ_name == 'nunu' or champ_name == 'renata':
                print(counter.group(0))
            champ_counters[champ]+= [counter.group(0)]


Wukong
Nunu
Gwen
Elise
Master Yi
Karthus
Bel'Veth
Briar
Brand
Talon
Rek'Sai
Hecarim
Volibear
Gragas
Viego
Fiddlesticks
Ekko
Lillia
Wukong
Amumu
Trundle
Zac
Ivern
Xin Zhao
Kindred
Lee Sin
Kayn
Rell
Sejuani
Udyr
Jax
Kha'Zix
Zed
Nidalee
Poppy
Shaco
Taliyah
Graves
Rengar
Jarvan IV
Warwick
Evelynn
Diana
Vi
Nocturne
Rammus
Maokai
Shyvana
Renata
Taric
Janna
Poppy
Twitch
Vel'Koz
Nami
Pyke
Senna
Soraka
Karma
Lulu
Bard
Zyra
Zilean
Brand
Seraphine
Milio
Maokai
Rell
Sona
Rakan
Xerath
Braum
Ashe
Blitzcrank
Camille
Lux
Yuumi
Shaco
Morgana
Pantheon
Neeko
Hwei
Leona
Thresh
Alistar
Nautilus
Zac
Swain
Tahm Kench
AurelionSol
Belveth
Chogath
DrMundo
JarvanIV
Kaisa
Khazix
KogMaw
KSante
LeeSin
MasterYi
MissFortune
RekSai
TahmKench
TwistedFate
XinZhao
Aatrox
Ahri
Akali
Akshan
Alistar
Amumu
Anivia
Annie
Ashe
Azir
Bard
Blitzcrank
Brand
Braum
Briar
Caitlyn
Camille
Cassiopeia
Corki
Darius
Diana
Draven
Ekko
Elise
Evelynn
Ezreal
Fiddlesticks
Fiora
Fizz
Galio
Gangplank
Garen
Gnar
Gragas
Graves
Gwen
Hecarim
Heimerdi

In [136]:
champ_counters

{'Wukong': ['Ivern',
  'Gwen',
  "Rek'Sai",
  'Rumble',
  'Taliyah',
  'Talon',
  'Karthus',
  'Volibear',
  'Fiddlesticks',
  'Evelynn',
  'Hecarim',
  'Udyr',
  'Nidalee',
  'Elise',
  'Graves',
  'Ekko',
  'Lee Sin',
  'Jax',
  'Jarvan IV',
  'Zed',
  'Rengar',
  "Bel'Veth",
  'Zac',
  'Gragas',
  'Lillia',
  'Brand',
  "Kha'Zix",
  'Xin Zhao',
  'Viego',
  'Amumu',
  'Rammus',
  'Vi',
  'Nunu & Willump',
  'Shaco',
  'Kindred',
  'Kayn',
  'Poppy',
  'Briar',
  'Skarner',
  'Master Yi',
  'Warwick',
  'Nocturne',
  'Trundle',
  'Shyvana',
  'Diana',
  'Rell',
  'Maokai',
  'Sejuani'],
 'Nunu': ['Gwen',
  'Elise',
  'Master Yi',
  'Karthus',
  "Bel'Veth",
  'Briar',
  'Brand',
  'Talon',
  "Rek'Sai",
  'Hecarim',
  'Volibear',
  'Gragas',
  'Viego',
  'Fiddlesticks',
  'Ekko',
  'Lillia',
  'Wukong',
  'Amumu',
  'Trundle',
  'Zac',
  'Ivern',
  'Xin Zhao',
  'Kindred',
  'Lee Sin',
  'Kayn',
  'Rell',
  'Sejuani',
  'Udyr',
  'Jax',
  "Kha'Zix",
  'Zed',
  'Nidalee',
  'Poppy',
  '

In [137]:
to_change = {
                    'Nunu' : 'Nunu & Willump',
                    'Renata' : 'Renata Glasc',
                    'AurelionSol': 'Aurelion Sol',
                    'Belveth' : 'Bel\'Veth',
                    'Chogath': 'Cho\'Gath',
                    'DrMundo': 'Dr. Mundo',
                    'JarvanIV': 'Jarvan IV',
                    'Kaisa': 'Kai\'Sa',
                    'Khazix': 'Kha\'Zix',
                    'KogMaw': 'Kog\'Maw',
                    'KSante': 'K\'Sante',
                    'LeeSin': 'Lee Sin',
                    'MasterYi': 'Master Yi',
                    'MissFortune': 'Miss Fortune',
                    'RekSai' : 'Rek\'Sai',
                    'TahmKench' : 'Tahm Kench',
                    'TwistedFate': 'Twisted Fate',
                    'XinZhao' : 'Xin Zhao',
                    'VelKoz': 'Vel\'Koz'}

for key in champs_names_list:
    if key in to_change.keys():
        champ_counters[to_change[key]] = champ_counters[key]
        del champ_counters[key]

In [138]:
champ_counters['Vel\'Koz']

['Zac',
 'Zilean',
 'Soraka',
 'Pyke',
 'Blitzcrank',
 'Yuumi',
 'Camille',
 'Taric',
 'Maokai',
 'Janna',
 'Rakan',
 'Shaco',
 'Zyra',
 'Bard',
 'Milio',
 'Xerath',
 'Morgana',
 'Rell',
 'Renata Glasc',
 'Leona',
 'Sona',
 'Seraphine',
 'Senna',
 'Twitch',
 'Brand',
 'Nami',
 'Ashe',
 'Karma',
 'Sylas',
 'Swain',
 'Lulu',
 'Nautilus',
 'Tahm Kench',
 'Lux',
 'Pantheon',
 'Hwei',
 'Alistar',
 'Thresh',
 'Neeko',
 'Braum']

In [139]:
json.dumps(champ_counters)

'{"Wukong": ["Ivern", "Gwen", "Rek\'Sai", "Rumble", "Taliyah", "Talon", "Karthus", "Volibear", "Fiddlesticks", "Evelynn", "Hecarim", "Udyr", "Nidalee", "Elise", "Graves", "Ekko", "Lee Sin", "Jax", "Jarvan IV", "Zed", "Rengar", "Bel\'Veth", "Zac", "Gragas", "Lillia", "Brand", "Kha\'Zix", "Xin Zhao", "Viego", "Amumu", "Rammus", "Vi", "Nunu & Willump", "Shaco", "Kindred", "Kayn", "Poppy", "Briar", "Skarner", "Master Yi", "Warwick", "Nocturne", "Trundle", "Shyvana", "Diana", "Rell", "Maokai", "Sejuani"], "Aatrox": ["Ahri", "Zed", "Qiyana", "Naafiri", "Karma", "Bel\'Veth", "Azir", "Singed", "Olaf", "Irelia", "Fiora", "Kled", "Akshan", "Cassiopeia", "Nocturne", "Quinn", "Rek\'Sai", "Malphite", "Maokai", "Garen", "Wukong", "Vayne", "Poppy", "Gwen", "Zac", "Warwick", "Heimerdinger", "Senna", "Kayle", "Jayce", "Jax", "Twisted Fate", "Riven", "Urgot", "Darius", "Gragas", "Volibear", "Renekton", "Sejuani", "Pantheon", "Rengar", "Ornn", "Briar", "Ryze", "Neeko", "Kennen", "Brand", "Camille", "Gnar